In [6]:
!pip uninstall torch torchvision torchaudio -y
!pip install torch torchvision torchaudio
!pip install datasets
!pip install transformers
!pip install sentencepiece
!pip install sacrebleu
!pip install evaluate
!pip install accelerate -U
!pip install gradio
!pip install kaleido cohere openai tiktoken

Found existing installation: torch 2.2.2
Uninstalling torch-2.2.2:
  Successfully uninstalled torch-2.2.2
Found existing installation: torchvision 0.17.2
Uninstalling torchvision-0.17.2:
  Successfully uninstalled torchvision-0.17.2
Found existing installation: torchaudio 2.2.2
Uninstalling torchaudio-2.2.2:
  Successfully uninstalled torchaudio-2.2.2
  Using cached torch-2.2.2-cp312-none-macosx_10_9_x86_64.whl.metadata (25 kB)
  Using cached torchvision-0.17.2-cp312-cp312-macosx_10_13_x86_64.whl.metadata (6.6 kB)
  Using cached torchaudio-2.2.2-cp312-cp312-macosx_10_13_x86_64.whl.metadata (6.4 kB)
Using cached torch-2.2.2-cp312-none-macosx_10_9_x86_64.whl (150.8 MB)
Using cached torchvision-0.17.2-cp312-cp312-macosx_10_13_x86_64.whl (1.7 MB)
Using cached torchaudio-2.2.2-cp312-cp312-macosx_10_13_x86_64.whl (3.4 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [torchaudio]3 [torchaudio]]


In [1]:
from datasets import load_dataset
dataset=load_dataset("cfilt/iitb-english-hindi")

README.md: 0.00B [00:00, ?B/s]

dataset_infos.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/190M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/85.7k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/500k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1659083 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/520 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [2]:
max_length=256
from transformers import AutoTokenizer,AutoModelForSeq2SeqLM
tokenizer=AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-hi")
model=AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-hi")

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
2026-01-24 13:36:00.179102: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


pytorch_model.bin:   0%|          | 0.00/306M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/306M [00:00<?, ?B/s]

In [3]:
article=dataset['validation'][2]['translation']['en']
inputs=tokenizer.encode(article,return_tensors='pt')
translated_tokerns=model.generate(inputs,max_length=max_length,num_beams=4,early_stopping=True)
tokenizer.batch_decode(translated_tokerns,skip_special_tokens=True)[0]

'एमएनएपी शिक्षकों के राष्ट्रपति, राजस्वीवर ने इस पुरस्कार को पेश करने के द्वारा स्कूल की प्रतिष्ठा की.'

In [4]:
dataset['validation'][2]['translation']['hi']  # reference translation

'मनपा शिक्षक संघ के अध्यक्ष राजेश गवरे ने स्कूल को भेंट देकर सराहना की।'

In [7]:
def preprocess_function(examples):
    # Handle nested translation structure properly
    translations = examples['translation']
    
    # When batched=True, translations is a list of dicts with 'en' and 'hi' keys
    inputs = [ex['en'] for ex in translations]
    targets = [ex['hi'] for ex in translations]
    
    model_inputs = tokenizer(inputs, max_length=max_length, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_length, truncation=True)
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

In [8]:
tokenized_datasets_validation = dataset['validation'].map(preprocess_function, batched=True, batch_size=2)
tokenized_datasets_test = dataset['test'].map(preprocess_function, batched=True, batch_size=2)

Map:   0%|          | 0/520 [00:00<?, ? examples/s]

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:3970: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [9]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [17]:
for parameter in model.parameters():
    parameter.requires_grad = False
num_layers_to_freeze = 6
for layer_index, layer in enumerate(model.model.encoder.layers):
    if layer_index < len(model.model.encoder.layers) - num_layers_to_freeze:
        for param in layer.parameters():
            param.requires_grad = False
num_layers_to_freeze = 10 
for layer_index, layer in enumerate(model.model.decoder.layers):
    if layer_index < len(model.model.decoder.layers) - num_layers_to_freeze:
        for param in layer.parameters():
            param.requires_grad = False

In [22]:
import evaluate
metric= evaluate.load("sacrebleu")
import numpy as np
def compute_metrics(eval_preds):
    preds,labels=eval_preds
    if isinstance(preds,tuple):
        preds=preds[0]
    decoded_preds=tokenizer.batch_decode(preds,skip_special_tokens=True)

    labels=np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels=tokenizer.batch_decode(labels,skip_special_tokens=True)
    
    decoded_preds=[pred.strip() for pred in decoded_preds]
    decoded_labels=[[label.strip()] for label in decoded_labels]
    result=metric.compute(predictions=decoded_preds,references=decoded_labels)
    result={"bleu":result["score"]}
    return result

In [24]:
import torch
device = torch.device("cpu")
model.to(device)
from transformers import Seq2SeqTrainer,Seq2SeqTrainingArguments
training_args=Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=False,
)

In [25]:
trainer=Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets_test,
    eval_dataset=tokenized_datasets_validation,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,)
trainer.train()

/var/folders/79/qf4s258d1979pqm6v_z63bmc0000gn/T/ipykernel_3058/3546962064.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer=Seq2SeqTrainer(


ValueError: fp16 mixed precision with MPS device requires a Pytorch >= 2.5.0